In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

from compression import Region, Comparator
from ipywidgets import interact, FloatSlider

# TODO remove unused imports which are only needed in the library file

%matplotlib notebook

In [ ]:
# TODO set up stuff like a custom data directory and temp directory here
# pass them into the functions

DATA_DIR = os.path.join(os.path.expanduser("~"), "data", "fits")
colourmap = plt.cm.viridis


In [ ]:
regions = {
#     "GALFACTS N4 Q full": Region.from_fits(
#         os.path.join(DATA_DIR, "GALFACTS_N4_cube_Q_DQA3.3.1.fits")
#     ),

#     "GALFACTS N4 Q 512x512": Region.from_fits(
#         os.path.join(DATA_DIR, "GALFACTS_N4_cube_Q_DQA3.3.1.fits"), 
#         size=(512, 512),
#         centre=(768, 4600)
#     ),

#     "GALFACTS N4 I full": Region.from_fits(
#         os.path.join(DATA_DIR, "GALFACTS_N4_cube_I_DQA3.3.1.fits")
#     ),

#     "GALFACTS N4 I 512x512": Region.from_fits(
#         os.path.join(DATA_DIR, "GALFACTS_N4_cube_I_DQA3.3.1.fits"), 
#         size=(512, 512),
#         centre=(768, 4600)
#     ),

#     "DEEP full": Region.from_fits(
#         os.path.join(DATA_DIR, "DEEP_5_IQU_cube.fits")
#     ),

#     "DEEP 512x512": Region.from_fits(
#         os.path.join(DATA_DIR, "DEEP_5_IQU_cube.fits"),
#         size=(512, 512),
#         centre=(2048, 2048)
#     ),

#     "supermosaic full": Region.from_fits(
#         os.path.join(DATA_DIR, "supermosaic.10.fits")
#     ),

#     "supermosaic 512x512": Region.from_fits(
#         os.path.join(DATA_DIR, "supermosaic.10.fits"),
#         size=(512, 512),
#         centre=(900, 2000)
#     ),
    
#     "DEEP cropped": Region.from_fits(
#         os.path.join(DATA_DIR, "cropped_512_512_10_3_DEEP_5_IQU.fits"),
#         stokes=0,
#         channel=3
#     ),
    
    "m81": Region.from_fits(
        os.path.join(DATA_DIR, "m81.fits")
    ),
}

In [ ]:
# comparator = Comparator.compare_algorithms(regions["DEEP cropped"], colourmap)
comparator = Comparator.compare_algorithms(regions["m81"], colourmap)

In [ ]:
comparator.show_plots(datasets=["image"])


In [ ]:
interact(comparator.show_images, size=FloatSlider(value=0.5, min=0, max=1, step=0.01, continuous_update=False));